In [27]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azure.ai.ml import command
from azure.ai.ml.entities import Environment

In [1]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.16.1
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, opencensus-ext-azure, opencensus-ext-logging, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
try:
    credential=DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential=InteractiveBrowserCredential()

In [5]:
credential.get_token("https://management.azure.com/.default")

AccessToken(token='eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IktRMnRBY3JFN2xCYVZWR0JtYzVGb2JnZEpvNCIsImtpZCI6IktRMnRBY3JFN2xCYVZWR0JtYzVGb2JnZEpvNCJ9.eyJhdWQiOiJodHRwczovL21hbmFnZW1lbnQuYXp1cmUuY29tIiwiaXNzIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvNmExZDJmOTYtOGNkZi00ZDFhLTk0M2QtN2I3M2Y0ZGZiYjZkLyIsImlhdCI6MTcyNDUyNjU3MCwibmJmIjoxNzI0NTI2NTcwLCJleHAiOjE3MjQ1MzE0ODYsImFjciI6IjEiLCJhaW8iOiJBWVFBZS84WEFBQUE4SUZ4bVJxc0grMTBDZGpENlNVbXp1Mml5UEU5VzlKNkFEWnl0Q1FYTng0Qm0wRUFwcnVrMDRZOTRLR1UyQ1JKM3pjeU9XWG5qUFFMMWo2d1Z0ckdzdW53dmhyZFpTMGlTZHdWclJsWnBQMEo1eFFSZXM3aEFMYndteUY5cDVzSjNCMHVCVk9OaE1Vc3Q2Z1ZuZWNZWEhwWXBvQjhESVh0bDBFeTd6Ymhad2s9IiwiYWx0c2VjaWQiOiIxOmxpdmUuY29tOjAwMDNCRkZERERDRjk0ODEiLCJhbXIiOlsicHdkIiwibWZhIl0sImFwcGlkIjoiNTc5MzU5ZWYtMDlhYi00Mjc3LTlhODQtNWVjZjJmMjA5ZmI0IiwiYXBwaWRhY3IiOiIyIiwiZW1haWwiOiJyaXRpc2hhZGhpa2FyaUBnbWFpbC5jb20iLCJmYW1pbHlfbmFtZSI6IkFkaGlrYXJpIiwiZ2l2ZW5fbmFtZSI6IlJpdGlzaCIsImdyb3VwcyI6WyI5MmVmZTA4Yi0zYmFiLTRkMGUtOGNjZi00MjA1OTBhZGUwZWEiXSwiaWRwIjoibGl2ZS5jb20iLCJp

In [4]:
mlClient=MLClient.from_config(credential=credential)

Found the config file in: /config.json


In [6]:
mlClient

MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f633a9face0>,
         subscription_id=18a1f27f-edf5-495e-9acb-753c93335294,
         resource_group_name=rg-dp100-labs,
         workspace_name=mlw-dp100-labs)

In [7]:
%%writefile src/diabetes-trainingEnv.py

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,roc_curve

print("Loading Data ...")
diabetes=pd.read_csv("diabetes.csv")

X=diabetes[
    ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure',
    'TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
].values
y=diabetes['Diabetic'].values

X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.30, random_state=0)
reg=0.01
print('Training a logistic regression model with regularization rate of', reg)
model=LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# Calculate the Accuracy
yHat=model.predict(X=X_test)
acc=np.average(yHat==y_test)
print(f"Accuracy:{acc}")

# Calculate AUC
yScores=model.predict_proba(X=X_test)
auc=roc_auc_score(y_true=y_test,y_score=yScores[:,1])
print("AUC: "+str(auc))

Writing src/diabetes-trainingEnv.py


In [10]:
returnedJobCurated=mlClient.jobs.create_or_update(
    job=command(
        code="src",
        command="python diabetes-trainingEnv.py",
        environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
        compute="aml-cluster",
        display_name="diabetes-train-curated-env",
        experiment_name="diabetes-training"
        )
    )  
amlURLCurated=returnedJobCurated.studio_url
print(amlURLCurated)

Uploading src (0.52 MBs): 100%|██████████| 519945/519945 [00:00<00:00, 8890803.44it/s]




https://ml.azure.com/runs/olive_gas_gjnb7ww17b?wsid=/subscriptions/18a1f27f-edf5-495e-9acb-753c93335294/resourcegroups/rg-dp100-labs/workspaces/mlw-dp100-labs&tid=6a1d2f96-8cdf-4d1a-943d-7b73f4dfbb6d


<h3> List Environments </h3>

In [26]:
envs=mlClient.environments.list()
for env in envs:
    print(env.name)
    print(env.description)
    print(env.tags)

AzureML-ACPT-pytorch-1.13-py38-cuda11.7-gpu


In [24]:
env=mlClient.environments.get(
    name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu",
    version=44
    )
print(env.description,env.tags)

An environment for tasks such as regression, clustering, and classification with Scikit-learn. Contains the Azure ML SDK and additional python packages. {'Scikit-learn': '0.24.1', 'OS': 'Ubuntu18.04', 'Training': ''}


<h3> Create a Custom Environment </h3>

In [29]:
%%writefile src/conda-env.yml
name: basic-env-cpu
channels:
- conda-forge
dependencies:
    - python=3.11
    - scikit-learn
    - pandas
    - numpy
    - matplotlib

Writing src/conda-env.yml


In [31]:
envDockerConda=Environment(
        image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
        conda_file="./src/conda-env.yml",
        name="docker-image-plus-conda-example",
        description="Environment created from a Docker Image plus Conda environment"
    )
mlClient.environments.create_or_update(environment=envDockerConda)

Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'docker-image-plus-conda-example', 'description': 'Environment created from a Docker Image plus Conda environment', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/18a1f27f-edf5-495e-9acb-753c93335294/resourceGroups/rg-dp100-labs/providers/Microsoft.MachineLearningServices/workspaces/mlw-dp100-labs/environments/docker-image-plus-conda-example/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/ritishadhikaricompute/code/Users/ritishadhikari', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f631eacffa0>, 'serialize': <msrest.serialization.Serializer object at 0x7f631eacdf30>, 'version': '1', 'conda

In [34]:
returnedJobCustom=mlClient.jobs.create_or_update(
    job=command(
        code="src",
        command="python diabetes-trainingEnv.py",
        compute="aml-cluster",
        environment="docker-image-plus-conda-example@latest",
        experiment_name="diabetes-training",
        display_name="diabetes-training-custom-env"
    )
)

amlURLCustom=mlClient.jobs.create_or_update(job=returnedJobCustom)
print(amlURLCustom)

pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
Readonly attribute status will be ignored in class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.JobService'>
Readonly attribute status will be ignored in class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.JobService'>


type: command
outputs:
  default:
    mode: rw_mount
    type: uri_folder
    path: azureml://datastores/workspaceartifactstore/ExperimentRun/dcid.careful_king_g97w8dbc5x
environment: azureml:docker-image-plus-conda-example:1
resources:
  instance_count: 1
  shm_size: 2g
component:
  name: careful_king_g97w8dbc5x
  display_name: diabetes-training-custom-env
  type: command
  outputs:
    default:
      type: uri_folder
      mode: rw_mount
  command: python diabetes-trainingEnv.py
  environment: azureml:/subscriptions/18a1f27f-edf5-495e-9acb-753c93335294/resourceGroups/rg-dp100-labs/providers/Microsoft.MachineLearningServices/workspaces/mlw-dp100-labs/environments/docker-image-plus-conda-example/versions/1
  code: azureml:/subscriptions/18a1f27f-edf5-495e-9acb-753c93335294/resourceGroups/rg-dp100-labs/providers/Microsoft.MachineLearningServices/workspaces/mlw-dp100-labs/codes/583cecb3-fdd1-43b8-b100-9507711869ce/versions/1
  creation_context:
    created_at: '2024-08-24T20:10:45.176590